<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/ACRNN/CRNNdeep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
learning_rate = 0.001
training_epochs = 60
batch_size = 100

In [3]:
mel_X_train, mel_X_test, mel_Y_train, mel_Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_more.npy", allow_pickle=True)

print(mel_X_train.shape)
print(mel_Y_train.shape)
print(mel_X_test.shape)
print(mel_Y_test.shape)

(683, 128, 128)
(683,)
(228, 128, 128)
(228,)


In [4]:
mel_X_train = np.expand_dims(mel_X_train, axis=-1)
mel_X_test = np.expand_dims(mel_X_test, axis=-1)

mel_Y_train = to_categorical(mel_Y_train, 7)
mel_Y_test = to_categorical(mel_Y_test, 7)    

mel_train_dataset = tf.data.Dataset.from_tensor_slices((mel_X_train, mel_Y_train)).batch(batch_size)
mel_test_dataset = tf.data.Dataset.from_tensor_slices((mel_X_test, mel_Y_test)).batch(batch_size)

In [23]:
def create_model():
    inputs = keras.Input(shape=(128, 128, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D()(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D()(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D()(conv3)

    trans = keras.layers.Permute((2,1,3))(pool3)
    reshape = keras.layers.Reshape((-1, 16*128))(trans) # 열 개수(freq축)*ch
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=False, dropout=0.5))(reshape)

    fc = keras.layers.Dense(128)(lstm)
    drop = keras.layers.Dropout(0.5)(fc)
    output = keras.layers.Dense(7)(drop)
    return keras.Model(inputs=inputs, outputs=output)

In [24]:
model = create_model()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 16, 16, 128)       0   

In [25]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [26]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [27]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [28]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

In [29]:
print('Learning started. It takes sometime.')
print('Epoch\tloss\t\ttrain acc\ttest acc')
for epoch in range(training_epochs):
    loss = 0.
    train_acc = 0.
    test_acc = 0.
    
    for images, labels in mel_train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables)) 
        loss += loss_fn(model, images, labels)
        train_acc += evaluate(model, images, labels)
    loss = loss / len(mel_train_dataset)
    train_acc = train_acc / len(mel_train_dataset)
    
    for images, labels in mel_test_dataset:
        test_acc += evaluate(model, images, labels)
    test_acc = test_acc / len(mel_test_dataset)  

    print('{}  \t{:.4f}  \t{:.4f}  \t{:.4f}'.format(epoch+1, loss, train_acc, test_acc))

print('Learning Finished!')
#55분 시작

Learning started. It takes sometime.
Epoch	loss		train acc	test acc
1  	1.9029  	0.2261  	0.2200
2  	1.8259  	0.3185  	0.4114
3  	1.6352  	0.3731  	0.3790
4  	1.5132  	0.4308  	0.4210
5  	1.3855  	0.4337  	0.4752
6  	1.2979  	0.4946  	0.4867
7  	1.2296  	0.4940  	0.5071
8  	1.1574  	0.5532  	0.5543
9  	1.0712  	0.5755  	0.6076
10  	0.9998  	0.6118  	0.5871
11  	0.9569  	0.6290  	0.6110
12  	0.9188  	0.6684  	0.6171
13  	0.8941  	0.6669  	0.6057
14  	0.8517  	0.6921  	0.6243
15  	0.8111  	0.7067  	0.6005
16  	0.7715  	0.7101  	0.6595
17  	0.7342  	0.7347  	0.6343
18  	0.7408  	0.7516  	0.6290
19  	0.6337  	0.7967  	0.7367
20  	0.5536  	0.8176  	0.6410
21  	0.5998  	0.8030  	0.6343
22  	0.5585  	0.8110  	0.7119
23  	0.4700  	0.8513  	0.7300
24  	0.3893  	0.8762  	0.7014
25  	0.4044  	0.8751  	0.7214
26  	0.3812  	0.8751  	0.6152
27  	0.4235  	0.8622  	0.6490
28  	0.3811  	0.8968  	0.7267
29  	0.3256  	0.9025  	0.7133
30  	0.3806  	0.8956  	0.6967
31  	0.3409  	0.8894  	0.6557
32  	0.2856

In [ ]:
conf_mat = [[0]*7 for i in range(7)] #mat[real_label]=predicted_label list

for images, labels in mel_test_dataset:
  logits = model(images, training=False)
  logits_max = tf.math.argmax(logits,1)
  labels_max = tf.math.argmax(labels,1)
  for i in range(len(logits)): conf_mat[labels_max[i]][logits_max[i]]+=1

for i in range(7): print(conf_mat[i])

[46, 0, 0, 0, 2, 0, 0]
[0, 24, 2, 0, 1, 5, 4]
[5, 0, 18, 1, 0, 1, 4]
[7, 2, 0, 7, 3, 2, 5]
[7, 0, 7, 1, 6, 0, 3]
[0, 4, 1, 1, 0, 37, 0]
[1, 4, 1, 0, 0, 0, 16]


In [ ]:
conf_mat_normal = []
for i in range(7):
  s = sum(conf_mat[i])
  conf_mat_normal.append(list(map(lambda x:(x/s)*100, conf_mat[i])))

label = ['anger','boredom','disgust','fear','happy','sad','neutral']
print('\t'+'\t'.join(label))
for i in range(7):
  print(label[i], end='')
  for j in range(7): print('\t%2.0f'%conf_mat_normal[i][j], end=' ')
  print()

	anger	boredom	disgust	fear	happy	sad	neutral
anger	96 	 0 	 0 	 0 	 4 	 0 	 0 
boredom	 0 	67 	 6 	 0 	 3 	14 	11 
disgust	17 	 0 	62 	 3 	 0 	 3 	14 
fear	27 	 8 	 0 	27 	12 	 8 	19 
happy	29 	 0 	29 	 4 	25 	 0 	12 
sad	 0 	 9 	 2 	 2 	 0 	86 	 0 
neutral	 5 	18 	 5 	 0 	 0 	 0 	73 
